In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import csv
import string

In [2]:
#extracting text from xml files
from xml.dom import minidom

xmldoc = minidom.parse('1.1.text.xml')
def getTokens(node):
    tokens = []
    if node is not None:
        if node.nodeType == node.TEXT_NODE:
            tokens.extend(node.data.split())
        elif node.nodeType == node.ELEMENT_NODE:
            for sub_array in [item.data.split() for item in node.childNodes]:
                tokens.extend(sub_array)
    return tokens

In [3]:
#extracting all enities 
all_entities = []
for entity in xmldoc.getElementsByTagName('entity'):
    entityWords = getTokens(entity)
    all_entities.append(entityWords)

#print(all_entities)
len(all_entities)

5259

In [4]:
def geTitle(nodelist):
    titles = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            titles.append(node.data)
    return ''.join(titles)

In [5]:
#extract all titles. 
all_titles = []
for title in xmldoc.getElementsByTagName('title'):
    title_text = geTitle(title.childNodes)
    all_titles.append(title_text.lower())
#print(all_titles)  
len(all_titles)

350

In [8]:
def getAbstractText(abstractNode):
    # Extract abstract text
    abstractElements = []
    for item in abstractNode.childNodes:
        # If plain text node
        if item.nodeType == item.TEXT_NODE:
            abstractElements.append(item.data)
        # If xml tag node
        elif item.nodeType == item.ELEMENT_NODE:
            for sub_array in [sub_item.data for sub_item in item.childNodes]:
                abstractElements.append(sub_array)
    return ''.join(abstractElements)


all_abstracts = []
for abstract in xmldoc.getElementsByTagName('abstract'):
    abstractText = getAbstractText(abstract)
    all_abstracts.append(abstractText)

#print(all_abstracts)
len(all_abstracts)
type(all_abstracts)

list

In [13]:
#350 titles + 350 abstracts
all_data = all_titles + all_abstracts

#print(all_data)
len(all_data)
#type(all_data)

700

In [10]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import itertools

In [14]:
#tokenize text
tokens = [word_tokenize(i) for i in all_data]
tokens = list(itertools.chain.from_iterable(tokens))

#print(tokens)
len(tokens)

43385

In [15]:
#remove stopwords, dublicated words and punctuation
#stop_words = set(stopwords.words('english')) 
punctuation = ['.', ',', '//', ':', ';', ')', '(', '%', '-']
#filtered_txt = [w for w in tokens if not w in stop_words]  
filtered_txt = [] 

#for w in tokens: 
#    if w not in stop_words and w not in punctuation: 
#        filtered_txt.append(w)
  
for w in tokens: 
    if w not in punctuation: 
        filtered_txt.append(w)

#remove dublicates
tokens = list(set(tokens))        
#print(tokens)
len(tokens)

5469

In [16]:
#write vocabulary into a csv file
with open('vocab.txt', 'w') as f:
    for item in tokens:
        f.write("%s\n" % item)

In [28]:
#convert tokens of the vocabulary into a dataframe.
df_vocabulary = pd.DataFrame(tokens)
df_vocabulary.columns = ['Word']
#print(df_vocabulary)
df_vocabulary.head(5)
len(df_vocabulary)

5469

# Fetching Word Embeddings

In [18]:
#get Global vectors and store them in the embeddings_dict
embeddings_dict = {}
f=open('C:/Users/CRS/Desktop/Thesis/Glove/glove.6B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], "float32")
    embeddings_dict[word] = vector

In [59]:
embeddings_dict['oral'] 

array([-4.0088e-01,  3.8661e-01, -5.3597e-01, -8.1736e-02,  3.7877e-01,
       -1.0792e-01, -1.6147e-01, -2.7286e-01,  1.2769e-01, -9.9632e-01,
       -4.9670e-01, -2.0537e-02, -6.3191e-01, -7.4420e-02, -1.1382e-01,
       -1.7873e-01, -2.3323e-01,  6.3002e-02,  3.7498e-01, -1.0195e+00,
       -5.2802e-01, -4.8008e-01,  1.7030e-01,  9.8061e-01, -1.0713e-02,
       -6.6502e-01,  1.8153e-02, -1.2372e+00, -1.8794e-01, -1.1094e-01,
       -1.5612e-01, -7.6409e-02, -2.1182e-01, -1.9177e-01, -9.7702e-02,
        1.3644e-01,  3.0259e-02, -2.0226e-01, -1.8277e-01, -9.2327e-02,
        2.7776e-01, -3.5437e-01, -4.6328e-01,  2.4546e-02,  4.7235e-01,
        4.1232e-01,  1.7078e-01,  1.6008e-01,  6.7490e-02,  3.9132e-01,
       -4.1970e-01,  5.5769e-01,  1.2453e-04,  2.7068e-01, -4.2873e-01,
       -1.6938e-01,  3.0346e-01, -3.3970e-01,  3.6246e-01,  1.1654e+00,
       -2.2694e-01, -4.7843e-01,  7.3229e-01,  1.3514e-01, -1.2169e-01,
       -2.5132e-01,  4.3905e-03,  2.9101e-01, -8.1070e-02,  5.96

In [19]:
print(len(embeddings_dict))

400000


In [20]:
#Convert dictionary into a DataFrame
#put an id for each word
embeddings_df = pd.DataFrame(list(embeddings_dict.items()), columns=['Word', 'Vector'])
term_id = range(len(embeddings_dict))
embeddings_df.insert(0, "Id", term_id, True) 
embeddings_df.head(5)

,Id,Word,Vector
0,0,the,"[0.04656, 0.21318, -0.0074364, -0.45854, -0.03..."
1,1,",","[-0.25539, -0.25723, 0.13169, -0.042688, 0.218..."
2,2,.,"[-0.12559, 0.01363, 0.10306, -0.10123, 0.09812..."
3,3,of,"[-0.076947, -0.021211, 0.21271, -0.72232, -0.1..."
4,4,to,"[-0.25756, -0.057132, -0.6719, -0.38082, -0.36..."


In [62]:
for i in range(len(embeddings_df)):
    if embeddings_df['Word'][i] == 'oral':
        print(embeddings_df['Vector'][i])

[-4.0088e-01  3.8661e-01 -5.3597e-01 -8.1736e-02  3.7877e-01 -1.0792e-01
 -1.6147e-01 -2.7286e-01  1.2769e-01 -9.9632e-01 -4.9670e-01 -2.0537e-02
 -6.3191e-01 -7.4420e-02 -1.1382e-01 -1.7873e-01 -2.3323e-01  6.3002e-02
  3.7498e-01 -1.0195e+00 -5.2802e-01 -4.8008e-01  1.7030e-01  9.8061e-01
 -1.0713e-02 -6.6502e-01  1.8153e-02 -1.2372e+00 -1.8794e-01 -1.1094e-01
 -1.5612e-01 -7.6409e-02 -2.1182e-01 -1.9177e-01 -9.7702e-02  1.3644e-01
  3.0259e-02 -2.0226e-01 -1.8277e-01 -9.2327e-02  2.7776e-01 -3.5437e-01
 -4.6328e-01  2.4546e-02  4.7235e-01  4.1232e-01  1.7078e-01  1.6008e-01
  6.7490e-02  3.9132e-01 -4.1970e-01  5.5769e-01  1.2453e-04  2.7068e-01
 -4.2873e-01 -1.6938e-01  3.0346e-01 -3.3970e-01  3.6246e-01  1.1654e+00
 -2.2694e-01 -4.7843e-01  7.3229e-01  1.3514e-01 -1.2169e-01 -2.5132e-01
  4.3905e-03  2.9101e-01 -8.1070e-02  5.9610e-01  6.2256e-02 -5.6080e-01
  6.2352e-01  3.2212e-01  5.0335e-01 -2.6570e-01  5.6315e-01 -3.6209e-01
  2.6874e-01  5.5533e-02  7.0896e-02 -2.5238e-01  4

In [23]:
#get glove word embeddings for the words in our vocabulary. 
# constract a dictionary which contains words in our vocabulary and their respective embedding from GloVe. 
vocab_dictionary = {}
vocab_words = []
test_words = []
vector_embeddings = []
for i in range (len(df_vocabulary)):
    if df_vocabulary['Word'][i] in embeddings_dict:
        vocab_words.append(df_vocabulary['Word'][i])
        vector_embeddings.append(embeddings_dict[df_vocabulary['Word'][i]])

#print(vocab_words)
#print(vector_embeddings)
len(vocab_words)
#type(vector_embeddings)

4011

In [25]:
#word_id = range(len(df_vocabulary))
word_vector_final= pd.DataFrame({'Word': vocab_words, 'Embedding Vector': vector_embeddings})     
word_vector_final.index = range(len(word_vector_final)) 
display(word_vector_final.head(5))
len(word_vector_final)

,Word,Embedding Vector
0,but,"[-0.0093601, 0.22789, -0.10275, 0.0010893, 0.2..."
1,reported,"[-0.21093, 0.51757, 0.042662, 0.013656, -0.577..."
2,groups,"[-0.12724, 0.27224, -0.055019, -0.0030393, -0...."
3,clustering,"[-0.49152, 0.83904, 0.48919, -0.53234, -0.0898..."
4,bootstrap,"[-0.54975, 0.13891, 0.58945, -0.24693, 0.30243..."


4011

In [27]:
#merge the words in your vocabulary file with the ID and embeddings from glove. 
#get word embedding of the words in the vocabulary by simply joininig the dataframes. 
vocabulary = pd.merge(df_vocabulary, embeddings_df, on='Word')
len(vocabulary)

4011

In [156]:
#vocabulary['Vector'][0]
len(vocabulary['Vector'][0])

300

In [58]:
vocabulary = vocabulary[['Id','Word', 'Vector']]
display(vocabulary.head(10))
type(vocabulary['Vector'])

,Id,Word,Vector
0,34,but,"[-0.0093601, 0.22789, -0.10275, 0.0010893, 0.2..."
1,293,reported,"[-0.21093, 0.51757, 0.042662, 0.013656, -0.577..."
2,503,groups,"[-0.12724, 0.27224, -0.055019, -0.0030393, -0...."
3,45209,clustering,"[-0.49152, 0.83904, 0.48919, -0.53234, -0.0898..."
4,79856,bootstrap,"[-0.54975, 0.13891, 0.58945, -0.24693, 0.30243..."
5,4640,valuable,"[0.34956, 0.24624, 0.20949, -0.15911, 0.3061, ..."
6,4968,preferred,"[-0.20157, 0.31404, 0.42666, -0.17512, 0.39641..."
7,136385,unordered,"[0.26952, 0.00056443, 0.086649, 0.37182, -0.04..."
8,35890,'95,"[-0.3863, 0.23094, -0.38055, -0.28722, 0.15539..."
9,23821,cf,"[0.053901, 0.64365, 0.046508, 0.037297, -0.183..."


pandas.core.series.Series

In [46]:
import pickle
vocabulary.to_pickle("GloVe.pkl")

In [47]:
data = pd.read_pickle('GloVe.pkl')

In [53]:
#print(len(vocabulary.values))
file2write=open("GloVe.txt",'w')
file2write.write(str(vocabulary.values))
file2write=open("GloVe.txt","r+") 
#file2write.close()
#file2write.read()
#print(file2write)


In [54]:
vocab = vocabulary.drop(columns=['Vector'])
display(vocab.head(5))

,Id,Word
0,34,but
1,293,reported
2,503,groups
3,45209,clustering
4,79856,bootstrap


In [55]:
vocab.to_csv('vocab.csv', index=False) 